# Multi-Agent System (AAI-520 Final Project)
---

Team Members (Group 4)
- Atul Aneja
- Juan Triana Martinez
- Mostafa Zamaniturk
- Ramesh Dhanasekaran
- Victoria Dorn

## Investment Research Agentic AI 

This nobebook walks through the creation of an autonomous multi-agent system designed for performing dynamic, self-improving investment research using LLMs, financial data, and structured agent workflows. Built with CrewAI and LangGraph to orchestrate specialist agents in a collaborative research pipeline.

Code can be found on [github](https://github.com/vdorn5/aai-520-final-project/tree/main)

---
### Inlined Code: `src/tools/news_client.py`

 - **Module path**: `src.tools.news_client`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

def get_company_news(company_name):
    url = f"https://newsapi.org/v2/everything?q={company_name}&sortBy=publishedAt&apiKey={NEWS_API_KEY}"
    response = requests.get(url)
    articles = response.json().get("articles", [])[:5]
    return [article["title"] + " - " + article["description"] for article in articles if article.get("description")]


---
### Inlined Code: `src/agents/news_analyst_agent.py`

 - **Module path**: `src.agents.news_analyst_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
from src.tools.news_client import get_company_news
from tools.news_analysis_tools import NewsAnalysisTools
# Create a News Analyst agent
news_analyst = Agent(
    role='News Analyst',
    goal='Analyze the latest news and market sentiment for a given company.',
    backstory=(
        'A specialist in processing and interpreting financial news. You can '
        'gauge market sentiment, identify key narratives, and detect significant '
        'events reported in the media using your advanced analysis pipeline.'
    ),
    tools=[NewsAnalysisTools.news_analysis_pipeline],
    verbose=True,
    allow_delegation=False
)

class NewsAnalystAgent:
    def __init__(self, llm):
        self.llm = llm
        self.agent = Agent(
            role="News Analyst",
            goal="Analyze recent news for company sentiment",
            backstory="Expert in parsing financial news and identifying relevant trends.",
            verbose=True,
            allow_delegation=False,
            tools=[],
            llm=self.llm
        )

    def analyze(self, company_name):
        news = get_company_news(company_name)
        prompt = f"Summarize the following news about {company_name}:\n\n" + "\n".join(news)
        response = self.llm.complete(prompt)
        return response


---
### Inlined Code: `src/tools/yfinance_client.py`

 - **Module path**: `src.tools.yfinance_client`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai.tools import tool
import yfinance as yf
# --- Tool for Basic Stock Data ---
@tool("Stock Ticker Data Tool")
def get_stock_data(ticker: str) -> dict:
    """A tool to get basic financial data for a given stock ticker."""
    # Implementation remains the same...
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        "longName": info.get("longName"),
        "sector": info.get("sector"),
        "industry": info.get("industry"),
        "marketCap": info.get("marketCap"),
        "trailingPE": info.get("trailingPE"),
        "forwardPE": info.get("forwardPE"),
        "dividendYield": info.get("dividendYield"),
        "fiftyTwoWeekHigh": info.get("fiftyTwoWeekHigh"),
        "fiftyTwoWeekLow": info.get("fiftyTwoWeekLow"),
        "regularMarketPrice": info.get("regularMarketPrice")
    }

# --- Tool for Financial Statements (for EarningsAnalyst) ---
@tool("Company Financial Statements Tool")
def get_financial_statements(ticker: str) -> str:
    """
    A tool to get the most recent annual financial statements (Income Statement,
    Balance Sheet, and Cash Flow) for a given stock ticker.
    """
    stock = yf.Ticker(ticker)
    
    # Fetch the most recent annual data
    income_statement = stock.income_stmt.iloc[:, 0]
    balance_sheet = stock.balance_sheet.iloc[:, 0]
    cash_flow = stock.cashflow.iloc[:, 0]
    
    # Format into a single string for the LLM
    report = f"""
    Income Statement:\n{income_statement.to_string()}\n
    Balance Sheet:\n{balance_sheet.to_string()}\n
    Cash Flow Statement:\n{cash_flow.to_string()}
    """
    return report

def get_yfinance_financials(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials
    return financials.to_string()

def get_analyst_recommendations(ticker):
    stock = yf.Ticker(ticker)
    return stock.recommendations.tail(3).to_string()


---
### Inlined Code: `src/agents/earnings_analyst_agent.py`

 - **Module path**: `src.agents.earnings_analyst_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
from tools.yfinance_client import get_stock_data, get_financial_statements
from tools.memory_tools import read_memory
# Create an Earnings Analyst agent
earnings_analyst = Agent(
  role='Earnings Analyst',
  goal='Analyze the financial statements of a company to assess its financial health and performance, considering any past analyses.',
  backstory=(
    'A meticulous financial analyst with a deep understanding of corporate finance. '
    'You specialize in dissecting income statements, balance sheets, and cash flow statements '
    'to uncover underlying trends and financial stability. You also consult past reports to see what has changed.'
  ),
  tools=[get_stock_data, get_financial_statements, read_memory], # Give this agent the read_memory tool
  verbose=True,
  allow_delegation=False
)
from src.tools.yfinance_client import get_yfinance_financials
from src.tools.yfinance_client import get_yfinance_financials, get_analyst_recommendations
import pandas as pd
import io
import json
from functools import lru_cache

@lru_cache(maxsize=50)
def get_financials_cached(ticker):
    """Cached version of financial fetcher to avoid repeated API calls."""
    return get_yfinance_financials(ticker)

class EarningsAnalystAgent:
    def __init__(self, llm):
        self.llm = llm

        # Create the agent with the tools
        self.agent = Agent(
            role="Earnings Analyst",
            goal="Review earnings reports and interpret key financial data.",
            backstory="Expert in financial analysis and equity research.",
            verbose=True,
            llm=self.llm
        )

    # Helper: parse financials
    def _parse_financials(self, financials_str):
        """Convert the yfinance .to_string() output into a DataFrame for quick extraction."""
        try:
            df = pd.read_csv(io.StringIO(financials_str))
            return df
        except Exception as e:
            print(f"Warning: Could not parse financials as CSV: {e}")
            # fallback simple parse
            lines = [line.strip() for line in financials_str.split("\n") if line.strip()]
            return lines   

    # Main analyze method
    def analyze(self, ticker):
        print(f"\n[+] Running Earnings Analysis for {ticker}...\n")

        # Fetch data directly
        financials = get_financials_cached(ticker)
        recommendations = get_analyst_recommendations(ticker)

        # Error Handling 
        if not financials:
            return f"No financial data available for {ticker}."
        if not recommendations:
            recommendations = "No analyst recommendations available."

        # Attempt to parse numeric insights (for potential future use)
        parsed_financials = self._parse_financials(financials)
        if isinstance(parsed_financials, pd.DataFrame):
            print(f"Parsed financials into DataFrame with shape: {parsed_financials.shape}")
        else:
            print(f"Financials parsed as text with {len(parsed_financials)} lines")
        
        # Structured LLM prompt 
        prompt = f"""
You are an Earnings Analyst. 
Review the financial report and provide a structured analysis in JSON format.

**Company:** {ticker}

**Instructions:**
Analyze the financial data and respond ONLY in valid JSON format with the following structure:
{{
    "revenue": "string - Total revenue with growth percentage if available",
    "net_income": "string - Net income with growth/decline percentage",
    "eps": "string - Earnings per share with comparison to previous period",
    "growth_comment": "string - Brief analysis of growth trends and key metrics",
    "analyst_sentiment": "string - Summary of analyst recommendations and sentiment",
    "overall_sentiment": "positive|neutral|negative - Overall investment sentiment"
}}

**Financial Data to Analyze:**
1. **Quantitative extraction** - Identify key figures:
   - Total Revenue
   - Net Income  
   - Earnings Per Share (EPS)
   - Year-over-year or quarter-over-quarter changes

2. **Qualitative interpretation** - Discuss:
   - Performance trends
   - Profitability indicators
   - Growth signals

3. **Analyst sentiment** - Consider these latest recommendations:
{recommendations}

**Raw financial data:**
{financials}

Respond with ONLY the JSON object, no additional text or formatting.
"""
        response = self.llm.complete(prompt)

        # Parse JSON response
        try:
            result = json.loads(response)
        except json.JSONDecodeError as e:
            print(f"Warning: Failed to parse LLM response as JSON: {e}")
            print(f"Raw response: {response[:200]}...")
            result = {
                "error": "Failed to parse JSON response", 
                "raw_output": response,
                "revenue": "Unable to extract",
                "net_income": "Unable to extract", 
                "eps": "Unable to extract",
                "growth_comment": "Analysis failed due to parsing error",
                "analyst_sentiment": "Unable to extract",
                "overall_sentiment": "neutral"
            }

        return result


---
### Inlined Code: `src/agents/market_analyst_agent.py`

 - **Module path**: `src.agents.market_analyst_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
from tools.fred_client import get_fred_data

# Create a Market Analyst agent
market_analyst = Agent(
    role='Macroeconomic Analyst',
    goal='Provide macroeconomic context for the stock analysis.',
    backstory=(
        'An economist with expertise in tracking and interpreting broad economic indicators. '
        'Your insights help frame the company\'s performance within the larger economic picture.'
    ),
    tools=[get_fred_data],
    verbose=True,
    allow_delegation=False
)


---
### Inlined Code: `src/agents/critic_agent.py`

 - **Module path**: `src.agents.critic_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from crewai import Agent
# Create the Critic Agent
critic_agent = Agent(
    role='Quality Assurance Critic',
    goal='Critique and refine the investment report to ensure it is comprehensive, accurate, and actionable.',
    backstory=(
        'A highly experienced investment strategist with a keen eye for detail. You are tasked with '
        'reviewing financial reports to identify any gaps, logical inconsistencies, or unsupported claims. '
        'Your feedback is crucial for producing a final, high-quality investment recommendation.'
    ),
    verbose=True,
    allow_delegation=False
)

class CriticAgent:
    def __init__(self, llm):
        self.llm = llm
        self.agent = Agent(
            role="Critic",
            goal="Evaluate and provide constructive feedback on investment reports",
            backstory="A strict but fair financial critic who checks quality and completeness.",
            verbose=True,
            allow_delegation=False,
            tools=[],
            llm=self.llm
        )

    def critique(self, report_text):
        prompt = f"""
You're a financial report evaluator. Critically evaluate the following investment research report. Assess it based on:

1. Relevance to the company
2. Clarity and conciseness
3. Factual correctness
4. Completeness of the analysis

Give a score from 1 to 10 and explain your reasoning.

Report:
{report_text}
"""
        return self.llm.complete(prompt)


---
### Inlined Code: `src/utils/llm_wrapper.py`

 - **Module path**: `src.utils.llm_wrapper`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

class LLMWrapper:
    def __init__(self):
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def complete(self, prompt):
        response = self.client.chat.completions.create(
            model="gpt-5-nano",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()

    def show_models(self):
        models = self.client.models.list()

        for model in models.data:
            print(model.id)

---
### Inlined Code: `src/agents/orchestrator_agent.py`

 - **Module path**: `src.agents.orchestrator_agent`  
 - **Reason**: This file was imported in the original notebook and has been inlined here to assist with submitting only one notebook file.
 - **Inlined on**: 2025-10-19


In [ ]:
from src.agents.news_analyst_agent import NewsAnalystAgent
from src.agents.earnings_analyst_agent import EarningsAnalystAgent
from src.agents.market_analyst_agent import MarketAnalystAgent
from src.agents.critic_agent import CriticAgent
from src.utils.llm_wrapper import LLMWrapper

class OrchestratorAgent:
    def __init__(self):
        self.llm = LLMWrapper()
        self.news_agent = NewsAnalystAgent(self.llm)
        self.earnings_agent = EarningsAnalystAgent(self.llm)
        self.market_agent = MarketAnalystAgent(self.llm)
        self.critic_agent = CriticAgent(self.llm)

    def run(self, ticker):
        print(f"\nStarting Investment Research for: {ticker}\n")

        print("News Summary:")
        news_summary = self.news_agent.analyze(ticker)
        print(news_summary)

        print("\nEarnings Summary:")
        earnings_summary = self.earnings_agent.analyze(ticker)
        print(earnings_summary)

        print("\nMarket Context:")
        market_summary = self.market_agent.analyze()
        print(market_summary)

        # Combine all sections into one report
        full_report = f"""
    # Investment Research Report: {ticker}

    ## News Summary:
    {news_summary}

    ## Earnings Summary:
    {earnings_summary}

    ## Market Context:
    {market_summary}
    """

        print("\nRunning Critique:\n")
        critique = self.critic_agent.critique(full_report)
        print(critique)

        print("\nFinal Investment Report with Critique Generated.\n")



In [ ]:
from src.agents.orchestrator_agent import OrchestratorAgent


In [ ]:
orchestrator = OrchestratorAgent()

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

models = client.models.list()

for model in models.data:
    print(model.id)

gpt-3.5-turbo
sora-2-pro
gpt-5-pro
gpt-audio-mini
gpt-audio-mini-2025-10-06
sora-2
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
o1-mini-2024-09-12
o1-mini
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-image-1
gpt-4o-

In [ ]:
orchestrator.run("TSLA")


Starting Investment Research for: TSLA

News Summary:
I don’t see the news text yet. Please paste the article or share the link you want summarized.

If you’d like, I can tailor the summary to your needs. For example:
- One-sentence summary
- 5–6 bullet points covering key takeaways (who, what, when, where, why, numbers)
- Brief analysis of potential impact on TSLA stock and the broader market

If you share the text, I’ll generate a concise, clear summary focused on TSLA.

Earnings Summary:

[+] Running Earnings Analysis for TSLA...

Parsed financials into DataFrame with shape: (47, 1)
{'revenue': '97.69B (+0.95% YoY)', 'net_income': '7.13B (-52.5% YoY)', 'eps': 'Diluted EPS 2.04 USD, down 52.6% YoY from 4.31 USD in 2023', 'growth_comment': 'Revenue expanded modestly to 97.69B (+0.95% YoY), while profitability deteriorated: gross profit declined from 17.66B to 17.45B and operating income fell from 8.89B to 7.76B as R&D and SG&A expenses rose. Net income dropped about 52.5% YoY, and di